In [2]:
import numpy as np
import cv2
import re
import os
import albumentations as A

In [3]:
img_dir = '/data/cv_project/spikedroid/ploid_classification'
mask_dir = '/home/jupyter-n.artemenko/projects/spikelet_pubescence/masks'
save_dir = '/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine'

### **Get image, mask, make crop and save**

In [4]:
# define regex patterns
ploid_patt = r'(Гексаплоиды)|(Диплоиды)|(Тетраплоиды)'
pin_patt = r'[\d]{2,5}_[^,(I|l)]_\{[XХI]+-?\d{2}\}\.(jpg|JPG)$'
spec_patt = r'T\.\s[a-z]*'
table_patt = r'[\d]{2,5}_\{[ХXI]+-?\d{2}\}\.(jpg|JPG)$'
name_patt = r'[\d_]*\{[ХXI]{1,2}-?\d{2,5}\}'

In [5]:
for dirpath, dirnames, filenames in os.walk(img_dir):
    for fn in filenames:
        # Get data of ploidness and imagename
        ploid = re.search(ploid_patt, dirpath)
        if ploid:
            ploid = ploid.group()
        spec = re.search(spec_patt, dirpath)
        if spec:
            spec = spec.group()
        img_type = 'pin'
        img_name = re.search(pin_patt, fn)
        if not img_name:
            img_type = 'table'
            img_name = re.search(table_patt, fn)
        
            
        if img_name:
            img_name = img_name.group()
            if os.path.exists((f'{save_dir}/{ploid}/{spec}/{img_type}/{img_name}').replace('.jpg', '.png').replace('.JPG', '.png')):
                continue
            # Create directory with species
            if not os.path.exists(f'{save_dir}/{ploid}/{spec}/{img_type}'):
                spec = spec.replace(' ', '\ ')
                os.system(f'mkdir {save_dir}/{ploid}')
                os.system(f'mkdir {save_dir}/{ploid}/{spec}')
                os.system(f'mkdir {save_dir}/{ploid}/{spec}/{img_type}')
            # if os.path.exists(f'{save_dir}/{ploid}/{spec}/{img_type}/{img_name}'.replace('.jpg', '.png').replace('.JPG', '.png')):
            #    continue
            
            
            # Open files, compute product of images
            raw_img = cv2.imread(dirpath + '/' + img_name)
            nm = (mask_dir + '/' + ploid + '/' + img_type + '/' + img_name).replace('.jpg', '.png').replace('.JPG', '.png')
            mask = cv2.imread(nm)
            if mask is None:
                print(nm, 'There is no mask')
                continue
            if mask.shape != raw_img.shape:
                print(nm, 'Dim probl')
                continue
                
            # print(dirpath + '/' + img_name)
            tmp = np.moveaxis(mask, 2, 0)
            tmp[0] = tmp[1]
            tmp[2] = tmp[1]
            tmp = np.moveaxis(mask, 2, 2)
            img = cv2.bitwise_and(raw_img, tmp)
            
            # Make crop and unsqueeze to square binary-power form
            original = img.copy()
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
            
            # Find contours, obtain bounding box, extract and save ROI
            cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            print(cv2.boundingRect(cnts[1]))
            # List with crops
            areas = []
            for c in cnts:
                x,y,w,h = cv2.boundingRect(c)
                areas.append((x, y, w, h, w*h))
            areas = sorted(areas, key=lambda tup: tup[4])
            x,y,w,h,_ = areas[-2]
            # print(areas[-1])
            cv2.rectangle(img, (x, y), (x + w, y + h), (36,255,12), 2)
            img = original[y:y+h, x:x+w] 
            
            # If you want to extend image to required size
#            x,y,_ = img.shape
#             if y < 512 and x >= 512:
#                 square = np.zeros((x, 512, 3))
            
#                 # Getting the centering position
#                 ax,ay = (x - img.shape[0])//2,(512 - y)//2
            
#                 # Pasting the 'image' in a centering position
#                 if y % 2 == 0:
#                     square[ax:x,ay:512-ay] = img
#                 else:
#                     square[ax:x,ay:511-ay] = img
            
#                 spec = spec.replace('\ ', ' ')
#                 cv2.imwrite((f'{save_dir}/{ploid}/{spec}/{img_type}/{img_name}').replace('.jpg', '.png').replace('.JPG', '.png'), square)
#             elif x < 512 and y >= 512:
#                 square = np.zeros((512, y, 3))
                
#                 # Getting the centering position
#                 ax,ay = (512 - x)//2,(y - img.shape[1])//2
            
#                 # Pasting the 'image' in a centering position
#                 if x % 2 == 0:
#                     square[ax:512-ax,ay:y] = img
#                 else:
#                     square[ax:511-ax,ay:y] = img
                    
#                 spec = spec.replace('\ ', ' ')
#                 cv2.imwrite((f'{save_dir}/{ploid}/{spec}/{img_type}/{img_name}').replace('.jpg', '.png').replace('.JPG', '.png'), square)
#             elif x < 512 and y < 512:
#                 square = np.zeros((512, 512, 3))
                
#                 # Getting the centering position
#                 ax,ay = (512 - x)//2,(512 - y)//2
            
#                 # Pasting the 'image' in a centering position
#                 if x%2 == 0 and y%2 == 0:
#                     square[ax:512-ax,ay:512-ay] = img
#                 elif x%2 == 0 and  y%2 != 0:
#                     square[ax:512-ax,ay:511-ay] = img
#                 elif x%2 != 0 and y%2 == 0:
#                     square[ax:511-ax,ay:512-ay] = img
#                 else:
#                     square[ax:511-ax,ay:511-ay] = img
                    
#                 spec = spec.replace('\ ', ' ')
#                 cv2.imwrite((f'{save_dir}/{ploid}/{spec}/{img_type}/{img_name}').replace('.jpg', '.png').replace('.JPG', '.png'), square)
#             else:
            spec = spec.replace('\ ', ' ')
            cv2.imwrite((f'{save_dir}/{ploid}/{spec}/{img_type}/{img_name}').replace('.jpg', '.png').replace('.JPG', '.png'), img)

### Check crop sizes

In [25]:
for dirpath, dirnames, filenames in os.walk(save_dir):
    for fn in filenames:
        img = cv2.imread(dirpath + '/' + fn)
        x,y,_ = img.shape
        if x < 512:
            square = np.zeros((512, y, 3))
            # Getting the centering position
            ax,ay = (512 - x)//2,(y - img.shape[1])//2
            
            # Pasting the 'image' in a centering position
            if x % 2 == 0:
                square[ax:512-ax,ay:y] = img
            else:
                square[ax:511-ax,ay:y] = img
            
            # Get data of ploidness and imagename
            ploid = re.search(ploid_patt, dirpath)
            if ploid:
                ploid = ploid.group()
            spec = re.search(spec_patt, dirpath)
            if spec:
                spec = spec.group()
            img_type = 'pin'
            img_name = re.search(pin_patt, fn)
            if not img_name:
                img_type = 'table'
            img_name = re.search(table_patt, fn)
            spec = spec.replace('\ ', ' ')
            # print(dirpath, fn)
            print((f'{save_dir}/{ploid}/{spec}/{img_type}/{fn}').replace('.jpg', '.png').replace('.JPG', '.png'))
            cv2.imwrite((f'{save_dir}/{ploid}/{spec}/{img_type}/{fn}').replace('.jpg', '.png').replace('.JPG', '.png'), square)

/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Тетраплоиды/T. carthlicum/table/7654_{II-20}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Тетраплоиды/T. carthlicum/table/7774_{X-19}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Тетраплоиды/T. carthlicum/table/7794_{X-19}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Тетраплоиды/T. carthlicum/table/7782_{X-19}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Тетраплоиды/T. carthlicum/table/7771_{X-19}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Тетраплоиды/T. carthlicum/pin/7833_3_{X-19}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Тетраплоиды/T. carthlicum/pin/7741_3_{X-19}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Тетраплоиды/T. dicoccoides/table/7298_{X-19}.png
/home/jupyter-

/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. petropavlovsk/table/6753_{IX-18}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. petropavlovsk/table/6736_{IX-18}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. petropavlovsk/table/6553_{II-17}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. petropavlovsk/table/6756_{IX-18}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. petropavlovsk/table/6555_{II-17}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. petropavlovsk/table/6556_{II-17}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. petropavlovsk/table/6754_{IX-18}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. petropavlovsk/table/6

### Single image sementation remake

In [8]:
save_dir = '/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. vavilovii/pin'
mask_dir = '/home/jupyter-n.artemenko/projects/spikelet_pubescence/masks/Гексаплоиды/pin'

for dirpath, dirnames, filenames in os.walk(img_dir + '/Гексаплоиды/T. vavilovii/pin/251-260'):
    for fn in filenames:
        # Open files, compute product of images
        ploid = re.search(ploid_patt, dirpath)
        if ploid:
            ploid = ploid.group()
        spec = re.search(spec_patt, dirpath)
        if spec:
            spec = spec.group()
        img_type = 'pin'
        img_name = re.search(pin_patt, fn)
        if img_name:
            img_name = img_name.group()
        else:
            continue
            
        raw_img = cv2.imread(dirpath + '/' + fn)
        print(img_name.replace('.jpg', '.png').replace('.JPG', '.png'))
        mask = cv2.imread(mask_dir + '/' + img_name.replace('.jpg', '.png').replace('.JPG', '.png'))
        if mask is None:
            print(nm, 'There is no mask')
            continue
        if mask.shape != raw_img.shape:
            print(nm, 'Dim probl')
            continue
                
        # print(dirpath + '/' + img_name)
        tmp = np.moveaxis(mask, 2, 0)
        tmp[0] = tmp[1]
        tmp[2] = tmp[1]
        tmp = np.moveaxis(mask, 2, 2)
        img = cv2.bitwise_and(raw_img, tmp)
            
        # Make crop and unsqueeze to square binary-power form
        original = img.copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
            
        # Find contours, obtain bounding box, extract and save ROI
        cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        # List with crops
        areas = []
        for c in cnts:
            x,y,w,h = cv2.boundingRect(c)
            areas.append((x, y, w, h, w*h))
        areas = sorted(areas, key=lambda tup: tup[4])
        x,y,w,h,_ = areas[-2]
        # print(areas[-1])
        cv2.rectangle(img, (x, y), (x + w, y + h), (36,255,12), 2)
        img = original[y:y+h, x:x+w]
        cv2.imwrite(f'/home/jupyter-n.artemenko/projects/spikelet_pubescence/test/{img_name}', raw_img)
        cv2.imwrite(f'/home/jupyter-n.artemenko/projects/spikelet_pubescence/test/mask_{img_name}', mask)
            
        # Extend to required size and resize
        x,y,_ = img.shape
        if y < 512 and x >= 512:
            square = np.zeros((x, 512, 3))
            
            # Getting the centering position
            ax,ay = (x - img.shape[0])//2,(512 - y)//2
            
            # Pasting the 'image' in a centering position
            if y % 2 == 0:
                square[ax:x,ay:512-ay] = img
            else:
                square[ax:x,ay:511-ay] = img
            
            cv2.imwrite(save_dir + '/' + (fn.replace('.jpg', '.png').replace('.JPG', '.png')), square)
            print(save_dir + '/' + fn)
        elif x < 512 and y >= 512:
            square = np.zeros((512, y, 3))
                
            # Getting the centering position
            ax,ay = (512 - x)//2,(y - img.shape[1])//2
            
            # Pasting the 'image' in a centering position
            if x % 2 == 0:
                square[ax:512-ax,ay:y] = img
            else:
                square[ax:511-ax,ay:y] = img
                    
            cv2.imwrite(save_dir + '/' + (fn.replace('.jpg', '.png').replace('.JPG', '.png')), square)
            print(save_dir + '/' + fn)
        elif x < 512 and y < 512:
            square = np.zeros((512, 512, 3))
                
            # Getting the centering position
            ax,ay = (512 - x)//2,(512 - y)//2
            
            # Pasting the 'image' in a centering position
            if x%2 == 0 and y%2 == 0:
                square[ax:512-ax,ay:512-ay] = img
            elif x%2 == 0 and  y%2 != 0:
                square[ax:512-ax,ay:511-ay] = img
            elif x%2 != 0 and y%2 == 0:
                square[ax:511-ax,ay:512-ay] = img
            else:
                square[ax:511-ax,ay:511-ay] = img
                
            cv2.imwrite(save_dir + '/' + (fn.replace('.jpg', '.png').replace('.JPG', '.png')), square)
            print(save_dir + '/' + fn)
        else:
            cv2.imwrite(save_dir + '/' + (fn.replace('.jpg', '.png').replace('.JPG', '.png')), img)

259_4_{Х-20}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. vavilovii/pin/259_4_{Х-20}.jpg
259_3_{Х-20}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. vavilovii/pin/259_3_{Х-20}.jpg
256_2_{Х-20}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. vavilovii/pin/256_2_{Х-20}.jpg
260_4_{Х-20}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. vavilovii/pin/260_4_{Х-20}.jpg
256_3_{Х-20}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. vavilovii/pin/256_3_{Х-20}.jpg
256_4_{Х-20}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. vavilovii/pin/256_4_{Х-20}.jpg
257_1_{Х-20}.png
/home/jupyter-n.artemenko/projects/spikelet_pubescence/crops_without_spine/Гексаплоиды/T. vavilovii/pin/257_1_{Х-20}.jpg
258_2_{Х-20}.png
/home/jupyter-n.a

### Crops for holdout

In [1]:
base_dir = '/home/jupyter-n.artemenko/projects/spikelet_pubescence/test/Holdout/'
save_path = f'{base_dir}spikelets/'
inp_path = f'/data/cv_project/spikedroid/опушение/'
mask_path = f'{base_dir}masks/'

pubesc_patt = r'опушением'

In [4]:
with open(f'{base_dir}holdout_annotation', 'w') as oup:
    for dirpath, dirnames, filenames in os.walk(inp_path):
        for fn in filenames:
            # define target
            target = 1 if re.search(pubesc_patt, dirpath) else 0
            # read image, mask and get pure image
            raw_img = cv2.imread(f'{dirpath}/{fn}')
            mask = []
            tmp_path = f'{mask_path}неопушенные/{fn}'.replace('jpg', 'png')
            if target == 1:
                tmp_path = f'{mask_path}опушенные/{fn}'.replace('jpg', 'png')
                mask = cv2.imread(tmp_path)
            else:
                mask = cv2.imread(tmp_path)
            if mask is None:
                print(tmp_path, target)
                continue
            tmp = np.moveaxis(mask, 2, 0)
            tmp[0] = tmp[1]
            tmp[2] = tmp[1]
            tmp = np.moveaxis(mask, 2, 2)
            img = cv2.bitwise_and(raw_img, tmp)
                              
            # Make crop and unsqueeze to square binary-power form
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
            
            # Find contours, obtain bounding box, extract and save ROI
            cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            # List with crops
            areas = []
            for c in cnts:
                x,y,w,h = cv2.boundingRect(c)
                areas.append((x, y, w, h, w*h))
            areas = sorted(areas, key=lambda tup: tup[4])
            x,y,w,h,_ = areas[-2]
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 0), 2)
                              
            res = img[y:y+h, x:x+w]
            cv2.imwrite(f'{save_path}{fn}', res)
            oup.write(f'{save_path}{fn};{target};test\n')